## This nb assisted with loading and then subsetting of the watermarked generation data for the human study

### Note: this script should be moved to/run from the same dir as the `utils` subdir lives in to work properly

In [ ]:
import os
from argparse import Namespace
from tqdm import tqdm
from statistics import mean

import numpy as np
import pandas as pd
import torch

from datasets import Dataset

import matplotlib.pyplot as plt

from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

import cmasher as cmr


from utils.io import read_json, read_jsonlines, write_lst_json, write_jsonlines, write_json

from utils.notebooks import filter_text_col_length

In [ ]:

ROOT_DIR = "/cmlscratch/jkirchen/watermarking-root"
INPUT_DIR = f"{ROOT_DIR}/input"
OUTPUT_DIR = f"{ROOT_DIR}/output"
JSONL_FILENAME = "gen_table_w_metrics.jsonl"
META_FILENAME = "gen_table_w_metrics_meta.json"


# run_name = "vicuna_lfqa_1000_600_nofilter"
# run_name = "test_vicuna_c4"
# run_name = "test_vicuna_essays"

# run_name = "vicuna_lfqa/vicuna_lfqa_600_0-25_2-0_0-7_1-0_eval"
run_name = "vicuna_lfqa/vicuna_lfqa_600_0-25_4-0_0-7_1-0_eval"

run_dir = f"{OUTPUT_DIR}/{run_name}"
data_path = f"{run_dir}/{JSONL_FILENAME}"
meta_path = f"{run_dir}/{META_FILENAME}"

In [ ]:
args = Namespace()
metadata = read_json(meta_path)
args.__dict__.update(metadata)

raw_rows = [ex for ex in read_jsonlines(data_path)]
ds = Dataset.from_list(raw_rows)
ds

In [ ]:
# for k,v in args.__dict__.items():
#     print(f"{k}: {v}")

### LFQA paraphrase setting

In [ ]:
# df = ds.to_pandas()

# # First filter setting
# filter_cols = {
#     "baseline_completion": (600, 600, 1000),
#     # "no_wm_output"       : (300, 100, 1000), # if this is not for para qual then okay
#     "w_wm_output"        : (300, 100, 1000)
# }
# count_suffix = "_length"
# count_suffix = "_num_tokens_scored"

# for col_name, (target_T, lower_tol, upper_tol) in filter_cols.items():
#     df = filter_text_col_length(df, 
#                                 text_col_name=col_name, 
#                                 count_suffix=count_suffix, 
#                                 upper_T=target_T+upper_tol, 
#                                 lower_T=target_T-lower_tol)
    
# text_col_names = ["truncated_input", "baseline_completion", "no_wm_output", "w_wm_output"]
# metric_col_names = ["baseline_completion_z_score", "w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_repetition_4", "w_wm_output_log_diversity"]

# # df[text_col_names + metric_col_names]

# # # First filtered setting
# # print(f"Metric filter drop:")
# # print(len(df))
# # df = df[df[metric_col_names[0]] < 3.0]
# # df = df[df[metric_col_names[2]] > 6.0]
# # # df = df[df[metric_col_names[3]] > 0.5] # doesn't yield 60 with the non-english drop
# # df = df[df[metric_col_names[3]] > 0.468] # target 60 w/ non-english drop
# # print(len(df))

# # # Filtered setting will full dataset as input
# # hparams = "0-25_2-0_0-7_1-0"
# # print(f"Metric filter drop:")
# # print(len(df))
# # df = df[df[metric_col_names[2]] > 7.5]
# # df = df[df[metric_col_names[3]] > 0.445] # target 60 w/ non-english drop
# # # df = df[df[metric_col_names[4]] < 0.15]
# # # df = df[df[metric_col_names[5]] > 4.5]
# # print(len(df))

# # Filtered setting will full dataset as input
# hparams = "0-25_4-0_0-7_1-0"
# print(f"Metric filter drop:")
# print(len(df))
# df = df[df[metric_col_names[2]] > 9.0]
# df = df[df[metric_col_names[3]] > 0.6]
# df = df[df[metric_col_names[4]] < 0.11] # target 60 w/ non-english drop
# # df = df[df[metric_col_names[5]] > 4.5]
# print(len(df))

# # check whether string contains non-english unicode characters
# def is_english(s):
#     try:
#         s.encode(encoding='utf-8').decode('ascii')
#     except UnicodeDecodeError:
#         # print(s)
#         return False
#     else:
#         return True
# print(f"Non-english drop:")
# df = df[df["w_wm_output"].apply(is_english)]
# print(len(df))


In [ ]:
# # sort by multiple fields, some ascending, some descending
# # sort_cols = ["w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence"]
# # sort_ascending = [False, False, False, False]
# sort_cols = ["w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp"]
# sort_ascending = [False, False]

# df.sort_values(by=sort_cols, ascending=sort_ascending, inplace=False)[text_col_names + metric_col_names]

In [ ]:
# # plot a quick roc curve to check the starting perf of the subset
# from sklearn import metrics
# baseline_stats = df["baseline_completion_z_score"].values
# w_wm_stats = df["w_wm_output_z_score"].values
# all_scores = np.concatenate([baseline_stats, w_wm_stats])

# baseline_labels = np.zeros_like(baseline_stats)
# attacked_labels = np.ones_like(w_wm_stats)
# all_labels = np.concatenate([baseline_labels, attacked_labels])

# fpr, tpr, thresholds = metrics.roc_curve(all_labels, all_scores, pos_label=1)
# roc_auc = metrics.auc(fpr, tpr)
# print(roc_auc)

# plt.figure(figsize=(4,4))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate\n(1 - Specificity)')
# plt.ylabel('True Positive Rate\n(Sensitivity)')
# plt.title('ROC Curve')
# plt.legend(loc="lower right")
# plt.tight_layout()
# plt.show()

In [ ]:
# df["w_wm_output_num_tokens_scored"].describe()

### LFQA quality

In [ ]:
df = ds.to_pandas()

# # First filter setting
# filter_cols = {
#     "baseline_completion": (600, 600, 1000),
#     "no_wm_output"       : (300, 100, 1000), 
#     "w_wm_output"        : (300, 100, 1000)
# }
# Second filter setting, consider possible triangle setuup
filter_cols = {
    "baseline_completion": (600, 600, 1000),
    # "baseline_completion": (300, 100, 1000),
    "no_wm_output"       : (300, 100, 1000), 
    "w_wm_output"        : (300, 100, 1000)
}
count_suffix = "_length"
count_suffix = "_num_tokens_scored"

for col_name, (target_T, lower_tol, upper_tol) in filter_cols.items():
    df = filter_text_col_length(df, 
                                text_col_name=col_name, 
                                count_suffix=count_suffix, 
                                upper_T=target_T+upper_tol, 
                                lower_T=target_T-lower_tol)
    
text_col_names = ["truncated_input", "baseline_completion", "no_wm_output", "w_wm_output"]
metric_col_names = ["baseline_completion_z_score", "no_wm_output_num_tokens_scored", "w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "no_wm_output_vs_w_wm_output_p_sp", "w_wm_output_coherence"]

# df[text_col_names + metric_col_names]

# # First filtered setting
# print(f"Metric filter drop:")
# print(len(df))
# df = df[abs(df[metric_col_names[1]]-df[metric_col_names[2]]) < 80]
# df = df[df[metric_col_names[3]] > 4.0]
# df = df[df[metric_col_names[5]] > 0.7]
# print(len(df))

# # # Filtered setting will full dataset as input
# hparams = "0-25_2-0_0-7_1-0"
# print(f"Metric filter drop:")
# print(len(df))
# df = df[abs(df[metric_col_names[1]]-df[metric_col_names[2]]) < 85] # target 3*60 w/ non-english drop
# df = df[df[metric_col_names[3]] > 4.0]
# df = df[df[metric_col_names[5]] > 0.7]
# print(len(df))

# # Filtered setting will full dataset as input
hparams = "0-25_4-0_0-7_1-0"
print(f"Metric filter drop:")
print(len(df))
# df = df[abs(df[metric_col_names[1]]-df[metric_col_names[2]]) < 80]
df = df[abs(df[metric_col_names[1]]-df[metric_col_names[2]]) < 50]
df = df[df[metric_col_names[3]] > 4.0]
# df = df[df[metric_col_names[5]] > 0.64] # target 3*60 w/ non-english drop NOTE SCRATCHING THIS
print(len(df))

# check whether string contains non-english unicode characters
def is_english(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        # print(s)
        return False
    else:
        return True
print(f"Non-english drop:")
df = df[df["no_wm_output"].apply(is_english)]
df = df[df["w_wm_output"].apply(is_english)]
print(len(df))

In [ ]:
# # sort by multiple fields, some ascending, some descending
# # sort_cols = ["w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence"]
# # sort_ascending = [False, False, False, False]
# sort_cols = ["w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp"]
# sort_ascending = [False, False]

# df.sort_values(by=sort_cols, ascending=sort_ascending, inplace=False)[text_col_names + metric_col_names]

In [ ]:
# idx = 2256

# print(f"{text_col_names[0]}:\n{df.loc[idx][text_col_names[0]]}")
# print(f"{text_col_names[1]}:\n{df.loc[idx][text_col_names[1]]}")
# print(f"{text_col_names[2]}:\n{df.loc[idx][text_col_names[2]]}")
# print(f"{text_col_names[3]}:\n{df.loc[idx][text_col_names[3]]}")

# print(f"{metric_col_names[0]}:\n{df.loc[idx][metric_col_names[0]]}")
# print(f"{metric_col_names[1]}:\n{df.loc[idx][metric_col_names[1]]}")
# print(f"{metric_col_names[2]}:\n{df.loc[idx][metric_col_names[2]]}")
# print(f"{metric_col_names[3]}:\n{df.loc[idx][metric_col_names[3]]}")

In [ ]:
# df["no_wm_output_num_tokens_scored"].describe()

In [ ]:
# df["w_wm_output_num_tokens_scored"].describe()

In [ ]:
# # plot a quick roc curve to check the starting perf of the subset
# from sklearn import metrics
# baseline_stats = df["baseline_completion_z_score"].values
# w_wm_stats = df["w_wm_output_z_score"].values
# all_scores = np.concatenate([baseline_stats, w_wm_stats])

# baseline_labels = np.zeros_like(baseline_stats)
# attacked_labels = np.ones_like(w_wm_stats)
# all_labels = np.concatenate([baseline_labels, attacked_labels])

# fpr, tpr, thresholds = metrics.roc_curve(all_labels, all_scores, pos_label=1)
# roc_auc = metrics.auc(fpr, tpr)
# print(roc_auc)

# plt.figure(figsize=(4,4))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate\n(1 - Specificity)')
# plt.ylabel('True Positive Rate\n(Sensitivity)')
# plt.title('ROC Curve')
# plt.legend(loc="lower right")
# plt.tight_layout()
# plt.show()

### C4 paraphrase setting

In [ ]:
# df = ds.to_pandas()

# filter_cols = {
#     "baseline_completion": (200, 200, 1000),
#     # "no_wm_output"       : (200, 25, 25), # if this is not for para qual then okay
#     "w_wm_output"        : (200, 25, 25)
# }
# count_suffix = "_length"
# count_suffix = "_num_tokens_scored"

# for col_name, (target_T, lower_tol, upper_tol) in filter_cols.items():
#     df = filter_text_col_length(df, 
#                                 text_col_name=col_name, 
#                                 count_suffix=count_suffix, 
#                                 upper_T=target_T+upper_tol, 
#                                 lower_T=target_T-lower_tol)
    
# text_col_names = ["truncated_input", "baseline_completion", "no_wm_output", "w_wm_output"]
# metric_col_names = ["baseline_completion_z_score", "w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence", "w_wm_output_repetition_2"]

# # df[text_col_names + metric_col_names]

# print(len(df))
# df = df[df[metric_col_names[0]] < 3.0]
# df = df[df[metric_col_names[2]] > 7.0]
# df = df[df[metric_col_names[3]] > 0.5]
# df = df[df[metric_col_names[5]] < 0.035]
# print(len(df))


In [ ]:
# # sort by multiple fields, some ascending, some descending
# # sort_cols = ["w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence"]
# # sort_ascending = [False, False, False, False]
# sort_cols = ["w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence"]
# sort_ascending = [False, False, False]

# df.sort_values(by=sort_cols, ascending=sort_ascending, inplace=False)[text_col_names + metric_col_names]

### Essay paraphrase setting

In [ ]:
# df = ds.to_pandas()

# filter_cols = {
#     "baseline_completion": (600, 600, 1000),
#     # "no_wm_output"       : (300, 100, 1000), # if this is not for para qual then okay
#     "w_wm_output"        : (600, 0, 1000)
# }
# count_suffix = "_length"
# count_suffix = "_num_tokens_scored"

# for col_name, (target_T, lower_tol, upper_tol) in filter_cols.items():
#     df = filter_text_col_length(df, 
#                                 text_col_name=col_name, 
#                                 count_suffix=count_suffix, 
#                                 upper_T=target_T+upper_tol, 
#                                 lower_T=target_T-lower_tol)
    
# text_col_names = ["truncated_input", "baseline_completion", "no_wm_output", "w_wm_output"]
# metric_col_names = ["baseline_completion_z_score", "w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_ppl", "w_wm_output_repetition_4"]

# # df[text_col_names + metric_col_names]

# print(len(df))
# df = df[df[metric_col_names[0]] < 2.0]
# df = df[df[metric_col_names[2]] > 8.0]
# # df = df[df[metric_col_names[3]] > 0.5]
# # df = df[df[metric_col_names[4]] < 6.0]
# df = df[df[metric_col_names[5]] < 0.1]
# print(len(df))

In [ ]:
# # sort by multiple fields, some ascending, some descending
# # sort_cols = ["w_wm_output_num_tokens_scored", "w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp", "w_wm_output_coherence"]
# # sort_ascending = [False, False, False, False]
# # sort_cols = ["w_wm_output_z_score", "baseline_completion_vs_w_wm_output_p_sp"]
# sort_cols = ["baseline_completion_vs_w_wm_output_p_sp","w_wm_output_z_score"]
# sort_ascending = [False, False]

# df.sort_values(by=sort_cols, ascending=sort_ascending, inplace=False)[text_col_names + metric_col_names]

## Writing for paraphrase data format

In [ ]:
# # Reformat, select keys, and
# # write the raw rows to a json file formatted as a list of dicts
# def select_keys(ex):

#     ex = {
#         "idx": ex["idx"],
#         "truncated_input": ex["truncated_input"].replace("Answer the following question in 200-300 words. Explain it like I\'m five.\n\nQ:","").replace("\nA:",""),
#         "baseline_completion": ex["baseline_completion"],
#         "no_wm_output": ex["no_wm_output"],
#         "w_wm_output": ex["w_wm_output"],
#     }

#     return ex

# final_ds = Dataset.from_pandas(df)
# annotation_data_to_write = [{"data": select_keys(ex)} for ex in final_ds]

# write_lst_json(annotation_data_to_write, f"{run_dir}/filtered_for_paraphrase_annotation.json")

# full_raw_annotation_data = [ex for ex in final_ds]
# write_jsonlines(full_raw_annotation_data, f"{run_dir}/filtered_for_paraphrase_annotation_full.jsonl")

## Writing for quality rating data format

In [ ]:
# # Reformat, select keys, and
# # write the raw rows to a json file formatted as a list of dicts

# import random
# # set seed

# seed = 123
# random.seed(seed)

# randomize_pairs = True
# # randomize_pairs = False

# def select_keys(ex, randomize_pairs=False):
#     ex = {
#         "idx": ex["idx"],
#         "truncated_input": ex["truncated_input"].replace("Answer the following question in 200-300 words. Explain it like I\'m five.\n\nQ:","").replace("\nA:",""),
#         "baseline_completion": ex["baseline_completion"],
#         "no_wm_output": ex["no_wm_output"],
#         "w_wm_output": ex["w_wm_output"],
#     }
#     cols_to_show = ["no_wm_output", "w_wm_output"]
#     if not randomize_pairs:
#         ex["seq_a"] = ex["no_wm_output"]
#         ex["seq_b"] = ex["w_wm_output"]
#         ex["mapping"] = {
#             "seq_a": "no_wm_output",
#             "seq_b": "w_wm_output",
#         }
#     else:
#         a_idx = random.choice([0, 1])
#         ex["seq_a"] = ex[cols_to_show[a_idx]]
#         ex["seq_b"] = ex[cols_to_show[1 - a_idx]]
#         ex["mapping"] = {
#             "seq_a": cols_to_show[a_idx],
#             "seq_b": cols_to_show[1 - a_idx],
#         }
#     return ex

# final_ds = Dataset.from_pandas(df)
# annotation_data_to_write = [{"data": select_keys(ex, randomize_pairs)} for ex in final_ds]

# write_lst_json(annotation_data_to_write, f"{run_dir}/filtered_for_quality_annotation_rand-{randomize_pairs}{f'-{seed}' if randomize_pairs else ''}.json")

# full_raw_annotation_data = [ex for ex in final_ds]
# write_jsonlines(full_raw_annotation_data, f"{run_dir}/filtered_for_quality_annotation_rand-{randomize_pairs}{f'-{seed}' if randomize_pairs else ''}_full.jsonl")

## Load and check some paraphrases and prefs

In [ ]:
# from utils.evaluation import load_detector

In [ ]:
args = Namespace()

annotation_name = "annotation_check"
ANNOTATION_DIR = f"{OUTPUT_DIR}/{annotation_name}"

args.__dict__.update(read_json(f"{ANNOTATION_DIR}/gen_table_w_metrics_meta.json"))

# detector = load_detector(args)

orig_paraphrase_data = [ex for ex in read_jsonlines(f"{ANNOTATION_DIR}/filtered_for_paraphrase_annotation_full.jsonl")]
# paraphrase_annotations = read_json(f"{ANNOTATION_DIR}/paraphrase_annotations.json")
paraphrase_annotations = read_json(f"{ANNOTATION_DIR}/paraphrase-tomg-john-dump-project.json")

orig_preference_data = [ex for ex in read_jsonlines(f"{ANNOTATION_DIR}/filtered_for_quality_annotation_rand-True-123_full.jsonl")]
# preference_annotations = read_json(f"{ANNOTATION_DIR}/preference_annotations.json")
preference_annotations = read_json(f"{ANNOTATION_DIR}/preference-tomg-john-dump-project.json")

In [ ]:
# idx = 0
# idx = 1
# idx = 3
# # orig_stats = detector.detect(paraphrase_annotations[idx]["data"]["w_wm_output"][:805])
# orig_stats = detector.detect(paraphrase_annotations[idx]["data"]["w_wm_output"])
# annotation_stats = detector.detect(paraphrase_annotations[idx]["annotations"][0]["result"][0]["value"]["text"][0])
# time = paraphrase_annotations[idx]["annotations"][0]["lead_time"]
# print("time taken: ",time/60, "mins")

# print(f"Original len:\n{orig_stats['num_tokens_scored']}")
# print(f"Original Z:\n{orig_stats['z_score']}")
# print(f"Annotation stats len:\n{annotation_stats['num_tokens_scored']}")
# print(f"Annotation stats Z:\n{annotation_stats['z_score']}")

In [ ]:
# preference_annotations

In [ ]:
# all_prefs = []
# conversion = {"left":"seq_a","right":"seq_b"}

# for row in preference_annotations:
#     row_mapping = row["data"]["mapping"]

#     for annotation in row["annotations"]:
#         for result in annotation["result"]:
#             side = result["value"]["selected"]
#             seq_type = conversion[side]
#             output_type = row_mapping[seq_type]
#             all_prefs.append(output_type)

# print(f"Rate that un-watermarked is preferred out of {len(all_prefs)} samples: ", sum([p == "no_wm_output" for p in all_prefs])/len(all_prefs))
# all_prefs

In [ ]:
# def extract_paraphrase_annotations(annotations):
#     # pull out the annotation text, the completed_by and updated_by and the "data.idx" field and store in a list of dicts

#     # for each annotation, pull out the text, the completed_by and updated_by and the "data.idx" field
#     # and store in a list of dicts
#     paraphrase_annotations = []
#     for row in annotations:
#         annotation_data = { "idx": row["data"]["idx"] }

#         for annotation in row["annotations"]:
#             for result in annotation["result"]:
#                 annotation_data["text"] = result["value"]["text"][0]
#                 annotation_data["completed_by"] = annotation["completed_by"]
#                 annotation_data["updated_by"] = annotation["updated_by"]

#         paraphrase_annotations.append(annotation_data)

#     return paraphrase_annotations


# # construct index of paraphrase annotations
# # with completed_by as key and annotation data as value
# paraphrase_annotations_by_user = {}

# for row in extract_paraphrase_annotations(paraphrase_annotations):
#     if row["completed_by"] not in paraphrase_annotations_by_user:
#         paraphrase_annotations_by_user[row["completed_by"]] = []
#     paraphrase_annotations_by_user[row["completed_by"]].append(row)

# # check that each user only annotated each idx once, i.e. per user, the idxs are unique
# for user, annotations in paraphrase_annotations_by_user.items():
#     idxs = [row["idx"] for row in annotations]
#     assert len(idxs) == len(set(idxs)), f"User {user} annotated the same idx more than once"


In [ ]:
# paraphrase_annotations_by_user

In [ ]:
# # for each user, make a copy of the orig_paraphrase_data and then
# # make an index of their annotations with "idx" as key and annotation data as value
# # then for any row where the idx is in the user's annotations, add the annotation data text
# # as the "w_wm_output_attacked" field

# full_dataset_per_user = {}

# for user, annotations in paraphrase_annotations_by_user.items():
#     user_annotations_by_idx = {
#         row["idx"]: row
#         for row in annotations
#     }
#     user_full_rows = []
#     for row in orig_paraphrase_data:
#         if row["idx"] in user_annotations_by_idx:
#             row_copy = row.copy()
#             # hack to get rid of windowlist
#             for key in row.keys():
#                 if "window_list" in key: row_copy.pop(key)
            
#             row_copy["w_wm_output_attacked"] = user_annotations_by_idx[row_copy["idx"]]["text"]
#             user_full_rows.append(row_copy)

#     full_dataset_per_user[user] = user_full_rows

# # print lens of all user datasets
# for user, user_full_rows in full_dataset_per_user.items():
#     print(f"{user}: {len(user_full_rows)}")

In [ ]:
# # write the full dataset per user to a json file formatted as a list of dicts
# for user, full_dataset in full_dataset_per_user.items():
#     user_dir = f"{ANNOTATION_DIR}/full_dataset_per_user/user_{user}"
#     os.makedirs(user_dir, exist_ok=True)
#     write_jsonlines(full_dataset, f"{user_dir}/gen_table_attacked.jsonl")
#     # write the args s metadata
#     write_json(args.__dict__, f"{user_dir}/gen_table_attacked_meta.json", indent=4)

In [ ]:
# # concat all the full datasets per user into a single dataset
# all_users_full_dataset = []
# for user, full_dataset in full_dataset_per_user.items():
#     all_users_full_dataset.extend(full_dataset)

# # print lens of all user datasets
# print(f"all_users_full_dataset: {len(all_users_full_dataset)}")

In [ ]:
# # write the full dataset per user to a json file formatted as a list of dicts

# user_dir = f"{ANNOTATION_DIR}/full_dataset_per_user/all_users"
# os.makedirs(user_dir, exist_ok=True)

# write_jsonlines(all_users_full_dataset, f"{user_dir}/gen_table_attacked.jsonl")
# # write the args as metadata
# write_json(args.__dict__, f"{user_dir}/gen_table_attacked_meta.json", indent=4)

In [ ]:
orig_stats = detector.detect("Imagine you are a big construction company and you want to build a big building on a big piece of land. But before you can build, you have to buy the land from the person who owns it.\n\nOne day, there was a small family living on that land for a long time without the land owner's permission. They built a small home and lived there for years and years, taking care of the land, planting flowers, and mowing the grass.\n\nOne day, when the big construction company wanted to buy the land, they found out the small family had been living there so long that they now actually owned the land, even though they had never officially bought it from the land owner.\n\nThis is called adverse possession. When someone uses someone else's land for a long time without the owner's permission, it is like that person is showing ownership and taking care of the land, it is like that person has a legal right to own the land, even though they didn't buy it from the land owner.\n\nThis is called squatters rights, it is when people move onto someone else's property without permission, and then they are able to legally own the land, it is like it is their own property.\n\nIn conclusion, when people live on a property without the land owner's permission for a long time, it is called squatters rights, it is when people show they are the rightful owner by using")
annotation_stats = detector.detect("Suppose you want to build a large building and you work for a firm that does construction.  You must acquire the land that you want before you're allowed to start construction.\n\nNow suppose that there is already a family living on that land without the permission of the person who owns it.  Even though they don't technically have the legal right to be there, they've been there a long time and have been doing a good job taking care of the land, doing upkeep, landscaping, etc.\n\nNow the construction company comes along and wants to buy the land so they can being construction. However, they find out that the family living there now owns the land, despite the fact that they never actually paid for it in money.  \n\nThis is what lawyers call \"adverse possession.\"  In some states, living in a place for long enough, and doing a good job taking care of the land, is enough to enable someone to take legal ownership of the land without formally buying it.\n\nSometimes we call this \"squatters rights.\" It refers to a situation where a person moves onto land without permission, or \"squats\" on the land, and eventually the land becomes their property because they have been there a long time.\n\nTo wrap up, a person can live on land for a long time, and then use a policy called \"squatters rights\" to declare that they own the land without official paying for it.")

print(f"Original len:\n{orig_stats['num_tokens_scored']}")
print(f"Original Z:\n{orig_stats['z_score']}")
print(f"Annotation stats len:\n{annotation_stats['num_tokens_scored']}")
print(f"Annotation stats Z:\n{annotation_stats['z_score']}")

In [ ]:
orig_stats = detector.detect("Well, it depends on the country and how they execute people. Some countries, like the United States, have the death penalty, meaning a person can be sentenced to death if they are convicted of a crime.\n\nIn the United States, it can take a long time between sentencing and execution for a few reasons.\nFirst, there is the possibility of appealing the decision. This means that someone who has been sentenced to death may be able to argue their case again, to see if there was any mistake made during the trial, or if there was new evidence that wasn't considered before.\nNext, there can be many legal proceedings that happen between sentencing and execution. For example, there may be a final appeal to the Supreme Court, which can take a long time to decide.\nThere can also be different stages of appeals depending on the state.\nAdditionally, there can be many legal motions that can take a long time like requesting additional appeal, DNA testing and many more things.\nIn conclusion, it is not uncommon for someone sentenced to death to not be executed for years, if not even decades.\nThis is different to some countries, that might have different system and way to execute people.\nSo it is worth to mention that death penalty is controversial topic and many people oppose it due to various reasons and it is not applied in many countries anymore.\nSo it is worth to mention that death penalty is controversial topic and many people oppose it due to various reasons and it is not")
annotation_stats = detector.detect("This varies from country to country but in the United States for example, if you have been convicted of a crime you can be sentenced to death, i.e. given the \"death penalty\".\n\nSometimes there can be a long delay between when you are sentenced and when you are actually executed in the United States. Some things that might delay the process are the presentation of new evidence or whether or not there was a mistake made during the trial. Additionally, you have the basic right to appeal the decision.\nThe appeals process can take a long time as the number of stages varies from state to state. Final appeals to the Supreme Court can take a particularly long time to decide. Other legal motions like requests for forensic testing can take additional time as well.\nAs a result, a total delay of years to decades is possible.\n\nBut this does vary a lot from country to country based on the specifics of their system regarding execution. You should note that this topic is quite controversial and as such in many countries there is public opposition to it, or it has been phased out of use entirely. It is worth mentioning this controversial nature.")

print(f"Original len:\n{orig_stats['num_tokens_scored']}")
print(f"Original Z:\n{orig_stats['z_score']}")
print(f"Annotation stats len:\n{annotation_stats['num_tokens_scored']}")
print(f"Annotation stats Z:\n{annotation_stats['z_score']}")